<b> Processing Toronto Wiki table </b>

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
!pip install folium
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
    100% |████████████████████████████████| 92kB 7.4MB/s eta 0:00:01
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from

In [101]:
#create an empty data frame
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
toronto = pd.DataFrame(columns=column_names)
toronto #check the df

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

#pulll out the table elements and fill out the dataframe, clean up of newline stings on the way
for items in soup.find_all('tr')[1::1]:
    data = items.find_all('td')
    try:
        post=data[0].text
        postcode =post.replace("\n","Not assigned") 
        bor=data[1].text
        borough =bor.replace("\n","Not assigned") 
        neigh=data[2].text
        neighbourhood = neigh.replace("\n", "");
    except IndexError:pass
    toronto = toronto.append({'PostalCode': postcode,
                              'Borough': borough,
                              'Neighborhood': neighbourhood}, ignore_index=True)

#check the table
    toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [111]:
#clean up the table per requirement
#1 remove 'Not assigned Boroughs'
toronto=toronto[(toronto.Borough != "Not assigned")]

toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [115]:
#2 group by postalcode and borough
toronto1=toronto.groupby(by=('PostalCode', 'Borough'))['Neighborhood'].apply(', '.join).reset_index()
toronto=toronto[(toronto.Borough != "BNot assigned")]#clean up remaining non assigned boroughs

toronto1.head(50)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [116]:
toronto1.Neighborhood.unique

<bound method Series.unique of 0                                         Rouge, Malvern
1                 Highland Creek, Rouge Hill, Port Union
2                      Guildwood, Morningside, West Hill
3                                                 Woburn
4                                              Cedarbrae
5                                    Scarborough Village
6            East Birchmount Park, Ionview, Kennedy Park
7                        Clairlea, Golden Mile, Oakridge
8        Cliffcrest, Cliffside, Scarborough Village West
9                            Birch Cliff, Cliffside West
10     Dorset Park, Scarborough Town Centre, Wexford ...
11                                     Maryvale, Wexford
12                                             Agincourt
13               Clarks Corners, Sullivan, Tam O'Shanter
14     Agincourt North, L'Amoreaux East, Milliken, St...
15                         L'Amoreaux West, Steeles West
16                                           Upper Rouge


In [118]:
#3 asign neighborhoods which are not assigned
toronto1['Neighborhood'] = np.where(toronto1['Neighborhood'] == 'Not assigned', toronto1['Borough'], toronto1['Neighborhood'])
                                                                                         
toronto1.Neighborhood.unique()

array(['Rouge, Malvern', 'Highland Creek, Rouge Hill, Port Union',
       'Guildwood, Morningside, West Hill', 'Woburn', 'Cedarbrae',
       'Scarborough Village',
       'East Birchmount Park, Ionview, Kennedy Park',
       'Clairlea, Golden Mile, Oakridge',
       'Cliffcrest, Cliffside, Scarborough Village West',
       'Birch Cliff, Cliffside West',
       'Dorset Park, Scarborough Town Centre, Wexford Heights',
       'Maryvale, Wexford', 'Agincourt',
       "Clarks Corners, Sullivan, Tam O'Shanter",
       "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
       "L'Amoreaux West, Steeles West", 'Upper Rouge', 'Hillcrest Village',
       'Fairview, Henry Farm, Oriole', 'Bayview Village',
       'Silver Hills, York Mills', 'Newtonbrook, Willowdale',
       'Willowdale South', 'York Mills West', 'Willowdale West',
       'Parkwoods', 'Don Mills North', 'Flemingdon Park, Don Mills South',
       'Bathurst Manor, Downsview North, Wilson Heights',
       'Northwood Park, York

In [119]:
toronto1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [121]:
toronto1.shape

(105, 3)